# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [3]:
data = pd.read_csv('data.csv')
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Два столбца типа float, пять типа int и пять типа object.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Количество значений в столбцах различается, значит в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о клиенте. Часть колонок описывает самого клиента: количество детей, семейное положение, возраст, уровень образования, пол. Остальные данные относятся к платёжеспобности клиента: имелось ли задолжность по возврату кредитов, ежемесячный доход клиента. Последний столбец содержит цель получения кредита.

Предварительно можно утверждать, что для исследования данных достаточно. Но в данных встречаются пропуски, а также дубликаты.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем сколько в таблице пропущенных значений:

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Итак, пропущенные значения содержат столбцы `days_employed` и `total_income`.

Методом isna() найдём все строки с пропусками в столбце `days_employed` и просмотрим первые десять.

In [5]:
data_out = data[data['days_employed'].isna()]
data_out.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Похоже, что пропуски в `days_employed` и в `total_income` содержатся в одних и тех же строках. Пропуски в этих столбцах имеют значение `NaN`. 

Предположительно пропущенные значения в столбце `days_employed` имеют полностью случайный характер, тогда вероятно наличие пропущенных значений в `total_income` зависит от пропущенных значений в `days_employed`.

Столбец `days_employed` –  общий трудовой стаж в днях не влияет на исследование, так что пропуски в нем пока можно оставить без изменений, а вот `total_income` - ежемесячный доход, важная характеристика, является количественной переменной. Пропуски в количественных переменных можно заполнить характерными значениями.

Всех клиентов можно разделить по типу занятости на следующие категории:

In [6]:
# тип занятости клиентов с пропущенными знчениями в столбце total_income
data_out['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

In [7]:
# посчитаем медиану значений "total_income" для каждой катеогрии занятости:
pensioner_med = data[data['income_type'] == 'пенсионер']['total_income'].median()
civil_servant_med = data[data['income_type'] == 'госслужащий']['total_income'].median()
companion_med = data[data['income_type'] == 'компаньон']['total_income'].median()
employee_med = data[data['income_type'] == 'сотрудник']['total_income'].median()
businessman_med =  data[data['income_type'] == 'предприниматель']['total_income'].median()

# Заполним пропуски 
data.loc[(data['income_type'] == 'пенсионер') & (data['total_income'].isna()), 'total_income'] = pensioner_med 
data.loc[(data['income_type'] == 'госслужащий') & (data['total_income'].isna()), 'total_income'] = civil_servant_med
data.loc[(data['income_type'] == 'компаньон') & (data['total_income'].isna()), 'total_income'] = companion_med
data.loc[(data['income_type'] == 'сотрудник') & (data['total_income'].isna()), 'total_income'] = employee_med   
data.loc[(data['income_type'] == 'предприниматель') & (data['total_income'].isna()),'total_income'] = businessman_med 

**Вывод**

В исходной таблице в столбцах `days_employed` и `total_income` мы обнаружили пропуски типа `NaN`. На данном этапе столбец `days_employed` оставили без изменений. Пропущенные значения в столбце `total_income` заполнили соответвующей медианой значений, предварительно посчитав ее для каждой категории занятости клиентов: 'пенсионер', 'госслужащий', 'компаньон', 'сотрудник', 'предприниматель'.

### Замена типа данных

Перед тем как сделать замену типа данных в столбце `'days_employed'`, заполним все пропущенные значения нулем:

In [8]:
data['days_employed'] = data['days_employed'].fillna(0)

Для замены типа данных воспользуемся методом `astype()` так как нам необходимо привести значения к целочисленному типу `('int')`:

In [9]:
data = data.astype({'days_employed': 'int', 'total_income': 'int'})

In [10]:
# data['days_employed'] = data['days_employed'].astype('int')
# data['total_income'] = data['total_income'].astype('int')

Выведем обновленную информацию о таблице:

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе метод `fillna` и `astype` можно применять к нескольким столбцам одновременно.
    
Если столбцов более 2–ух, то рекомендую применять цикл `for`.
    
Пример: 
    
`df.astype({ "col1": "float64", "col2": "int64"})`
    
</div>

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы заменили тип данных float в столбцах `'days_employed'` и `'total_income'` на тип `int`. Теперь в таблице: семь значений типа `int` и пять значений типа `object`.

### Обработка дубликатов

In [12]:
# найдем явные дубликаты с помощью метода duplicated() и подсчитаем их количество
print(data.duplicated().sum())

54


In [13]:
# с помощью метода drop_duplicates() удалим из датафрейма явные дубликаты и обновим индексы
data = data.drop_duplicates().reset_index(drop=True)

При первом обзоре данных в столбце `education` были видны строки с одинаковыми значениями, но в разных регистрах. Посмотрим список уникальных значений в столбце `education`:

In [14]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Действительно, строки в столбце `education` различаются регистром, это могут быть неявные дубликаты. Чтобы учесть такие дубликаты с помощью метода `str.lower()` приведем значения к нижнему регистру:

In [15]:
data['education'] = data['education'].str.lower()
data['education'].unique()

# с помощью метода duplicated() проверим наличие дубликатов и подсчитаем их количество
print(data.duplicated().sum())

17


In [16]:
# удалим новые дубликаты
data = data.drop_duplicates().reset_index(drop=True)

**Вывод**

В исходном наборе данных мы обнаружили и устранили явные и неявные дубликаты. 

### Лемматизация

In [17]:
# Найдем уникальные значения в столбце `purpose`
purpose_unique = data['purpose'].unique()
print(purpose_unique)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Выделим леммы в значениях столбца с целями получения кредита и найдем количество их повторений, для этого подключим библиотеки `pymystem3` и специальный контейннер `Counter`: 

In [18]:
lemma_list = [] 
m = Mystem()

for purpose in purpose_unique:
    lemmas = ''.join(m.lemmatize(purpose)).strip() # склеили результат лематизации
    lemma_list.append(lemmas)    # добавим в список лемматизированные значения purpose_unique
 
print(Counter(lemma_list))     # Подсчет числа упоминаний 

Installing mystem to /Users/visnazugan/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


Counter({'автомобиль': 2, 'покупка жилье': 1, 'приобретение автомобиль': 1, 'дополнительный образование': 1, 'сыграть свадьба': 1, 'операция с жилье': 1, 'образование': 1, 'на проведение свадьба': 1, 'покупка жилье для семья': 1, 'покупка недвижимость': 1, 'покупка коммерческий недвижимость': 1, 'покупка жилой недвижимость': 1, 'строительство собственный недвижимость': 1, 'недвижимость': 1, 'строительство недвижимость': 1, 'на покупка подержать автомобиль': 1, 'на покупка свой автомобиль': 1, 'операция с коммерческий недвижимость': 1, 'строительство жилой недвижимость': 1, 'жилье': 1, 'операция со свой недвижимость': 1, 'заниматься образование': 1, 'сделка с подержанный автомобиль': 1, 'получение образование': 1, 'свадьба': 1, 'получение дополнительный образование': 1, 'покупка свой жилье': 1, 'операция с недвижимость': 1, 'получение высокий образование': 1, 'свой автомобиль': 1, 'сделка с автомобиль': 1, 'профильный образование': 1, 'высокий образование': 1, 'покупка жилье для сдача':

**Вывод**

В столбце с целью кредита клиента длинные и различные формулировки, это во первых усложняет работу с данными, а во вторых не может давать реальную оценку по количеству клиентов с одинаковыми целями по смыслу, но с различной формулировкой. Поэтому будет правильно столбец `purpose` разделить на категории. С помощью лемматизации мы определили принцип категоризации, а именно можно выделить четыре категории целей взятия кредита: 'недвижимость', 'автомобиль', 'образование', 'свадьба'.

### Категоризация данных

In [19]:
# Запишем правило классиффикации клиентов как функцию: 
# на вход функции попадает вся строка датафрейма, а возвращает она категорию цели кредита клиента:
def create_category_purpose(row):
# """"
# Составляет список лимматизированных слов цели lem_purpose.
# Возвращает категорию цели кредита по значению найденных лемм, используя правила:
# - 'атомобиль', если 'атомобиль' содержится в значении lem_purpose;
# - 'недвижимость', если 'жилье' или 'недвижимость' содержится в значении lem_purpose;
# - 'свадьба', если 'свадьба' содержится в значении lem_purpose;
# - 'образование' во всех других случаях.
# """"
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
        return 'недвижимость'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    return 'образование'


# применим функцию и сохраним результат в новом столбце 'purpose_category':
data['purpose_category'] = data.apply(create_category_purpose, axis=1)    

In [20]:
# применим функцию value_counts() для подсчета значений каждой категории
data['purpose_category'].value_counts() 

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Категоризация переменной childe:

In [21]:
def children_status(row):
    children = row['children']
    if children > 0:
        return 'есть дети'
    return 'нет детей'
data['children_status'] = data.apply(children_status, axis=1)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_status
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,есть дети
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,нет детей
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,есть дети
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,есть дети


**Вывод**

Итак, мы классифицировали цели кредита клиента на 4 категории: 'недвижимость', 'автомобиль', 'образование', 'свадьба'. Теперь легко видеть какие у клиентов цели кредита. Также категоризация позволяет рассчитать количество клиентов в каждой категории, например больше всего клиентов хотят взять кредит на недвижимость.

Также мы выделили еще две категории клиентов: 'есть дети', 'нет детей', результат добавили в столбец `children_status`.  

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
# создадим сводную таблицу в которую выделим статус наличия детей и статус наличия задожности у клиента
data_pivot_children = data.pivot_table(index='children_status', columns = 'debt', values='days_employed', aggfunc='count')

# процент клиентов с детьми имеющих задолжность
data_pivot_children['percent_1'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1]) * 100
data_pivot_children

debt,0,1,percent_1
children_status,,,
есть дети,6639,677,9.253691
нет детей,13074,1064,7.525817


In [23]:
data.groupby('children')['debt'].count()

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

In [24]:
data_pivot_children = data.pivot_table(index='children', columns = 'debt', values='days_employed', aggfunc='count')
data_pivot_children['percent_1'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1]) * 100
data_pivot_children

debt,0,1,percent_1
children,,,
-1,46.0,1.0,2.127660
0,13028.0,1063.0,7.543822
1,4364.0,444.0,9.234609
2,1858.0,194.0,9.454191
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN
20,68.0,8.0,10.526316


**Вывод**

- Процент клиентов с детьми имеющих задолжность выше, чем у клиентов без детей. 
- Большинство клиентов, которые берут кредит не имеют детей.

Большинство кредитов берут клиенты с одним ребенком, однако процент невыплаты кредита в срок больше у клиентов с 4-мя детьми и с 20-тью. Также стоит заметить что нет данных в столбце наличия зодолжности у клиентов с 5-тью детьми.
Можно сделать вывод, что действительно зависимость между наличием детей и возвратом кредита в срок имеется.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
# построим сводную таблицу для установки зависимости между семейным положением и возвратом кредита в срок
data_pivot_family = data.pivot_table(index='family_status', columns='debt', values='days_employed', aggfunc='count')
# процент клиентов с детьми имеющих задолжность
data_pivot_family['percent_1'] = data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1]) * 100
data_pivot_family

debt,0,1,percent_1
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


**Вывод**

Явной зависимости между семейным положением и возвратом кредита в срок нет.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для того чтобы установить зависимость между уровнем дохода и возвратом кредита в срок создадим сводную таблицу, предварительно разделив данные на категории по уровню дохода.

In [26]:
def income_level(row):
    income = row['total_income']
    if income <= 20000:
        return 'бедность'
    if (income >20000) and (income <= 60000):
        return 'средний достаток'
    return 'богатые'

data['income_level'] = data.apply(income_level, axis=1)
print(data['income_level'].unique())

['богатые' 'средний достаток']


Получилось, что в наборе данных клиенты разделились только на две категории: 'богатые' и 'средний достаток'.

In [27]:
# построим сводную таблицу 
data_pivot_income = data.pivot_table(index='income_level', columns='debt', values='days_employed', aggfunc='count')
data_pivot_income['percent_1'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1]) * 100
data_pivot_income

debt,0,1,percent_1
income_level,,,
богатые,18956,1692,8.194498
средний достаток,757,49,6.079404


**Вывод**

Процент, имеющих задолжность, выше среди клиентов с доходом выше среднего, т.е. клиенты с более высоким доходом с большей вероятностью не вернут кредит в срок. Также интересно, что отсутвуют клиенты с доходом ниже среднего.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
data_pivot_purpose = data.pivot_table(index='purpose_category', columns='debt', values='days_employed', aggfunc='count')
data_pivot_purpose['percent_1'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1]) * 100
data_pivot_purpose    

debt,0,1,percent_1
purpose_category,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**

Несмотря на то, что категория цели кредита `'недвижимость'` наиболее крупная, процент клиентов имеющих задолжность в этой категории меньше чем в остальных. Наибольший процент клиентов не вернувших кредит в срок в категории `'автомобиль'`.

## Шаг 4. Общий вывод

Мы рассмотрели набор данных и выяснили, что наличие детей у клиента, а также их количество влияют на факт погашения кредита в срок, а вот семейное положение не имеет явного влияния. 